# Modelamiento Computacional Aplicado
## Modelo Depredador - Presa (Zorros y Conejos)

#### Alumno: Joaquín De Ferrari
#### Profesor: Claudio Torres <br>
<b>Fecha de presentación: 04-10-2022<b><br><br>


<b>Objetivo:</b><br><br> Crear un programa que simule el comportamiento de dos grupos de animales, los depredadores y las presas. La idea es que el comportamiento de las poblaciones se acerque al propuesto según el modelo de Lotka-Volterra.<br><br>


<b>Características generales:</b><br><br>
-Grilla discreta con condiciones de borde periódicas.<br>
-Animales se ubican dentro de las casillas de esta grilla.<br>
-Sólo puede haber un animal por casilla.<br>
-El comportamiento de los animales es individualizado: se itera para definir qué hace cada uno.<br>
-Los "ticks" representan el paso del tiempo y definen cuántas iteraciones se haran en total.<br>
-Se asume "pasto infinito".<br><br>

<b>Conejos (familias de conejos):</b><br><br>
-Representados por una lista: \[Tipo, Timer, Edad\].<br>
-Se reproducen tras una cantidad definida de ticks, según el "Timer" que aumenta en cada iteración.<br>
-Se mueven a una casilla adyacente aleatorio en cada iteración.<br>
-Se mueren si superan una edad definida (pero en general mueren siendo cazados).<br><br>

<b>Zorros:</b><br><br>
-Representados por una lista: \[Tipo, Timer, Edad, Dirección_Predominante\].<br>
-Se reproducen con cierta probabilidad cuando se alimentan.<br>
-Detectan conejos cercanos y se mueven hacia ellos rápidamente (pueden "saltarse" casillas).<br>
-Tienen una probabilidad de lograr alimentarse al estar cerca de una familia de conejos. La reproducción representa una "separación" del grupo original.<br>
-Tienen una probabilidad de eliminar a la familia de conejos al alimentarse.<br>
-Tras alimentarse tienen que esperar una cantidad de ticks para volver a hacerlo (cooldown).<br>
-Si no se alimentan tras una cierta cantidad de ticks (según "Timer"), se mueren.<br>
-Si no detectan conejos cerca, se mueven en la "direccion predominante" con cierta probabilidad, de lo contrario se mueven aleatoriamente.<br>
-Si tienen otros zorros adyacentes, pueden "pelearse" con ellos con cierta probabilidad, matándolos.<br>
-Se mueren si superan una edad definida (pero en general mueren de inanición o peleando).<br>



In [1]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
import random

random.seed(10)

# Parametros generales
GRID_SIZE = 100       # (int) Indica cuántas casillas tendrá la grilla cuadrada. Casilla -> 40000 m^2, Grilla -> 400 km^2
TICK_RATE = 0.01      # (float) Indica qué tan seguido se generan nuevos "cuadros" (imágenes) cuando se visualiza la grilla como animación
TOTAL_TICKS = 5000    # (int) Cuántas iteraciones tendrá el loop principal. 1 tick = 1/4 día = 6 horas
VISUALIZAR = True     # (bool) Si es True, se muestra la animación de la grilla en la pantalla (más lento)

# Parametros de conejos (o familias de conejos)
NUM_INICIAL_CONEJOS = 200   # (int) Cuántos conejos se ponen inicialmente en la grilla
VIDA_CONEJO =  10000        # (int) Cuántos ticks puede sobrevivir una familia de conejos como máximo
FREC_REP_CONEJO = 400       # (int) Cada cuántos ticks se reproduce una familia. 400 ticks = 100 dias
DIST_REP_CONEJO = 1         # (int) A cuántas casillas de distancia se generan las nuevas familias
PROB_REP_LEJANA = 0.0       # (float entre 0 y 1) Probabilidad de que al reproducirse, la familia se genere a una distancia el triple de lo normal

# Parametros de zorros
NUM_INICIAL_ZORROS = 1      # (int) Cuántos zorros se ponen inicialmente en la grilla
VIDA_ZORRO = 10000          # (int) Cuántos ticks puede sobrevivir un zorro como máximo
FREC_ALI_ZORRO = 28         # (int) Cuántos ticks pasan antes de que un zorro se muera por inanición. 28 ticks = 7 dias
PROB_REP_ZORRO = 0.15       # (float entre 0 y 1) Probabilidad de que un zorro se reproduzca al alimentarse
DIST_MOV_ZORRO = 4          # (int) Cuántas casillas puede moverse un zorro en cada tick. 1.2 km cada 6 horas
PROB_MOV_RAND = 0.1         # (float entre 0 y 1) Probabilidad de que un zorro se mueva de manera aleatoria y no en su dirección predominante
DIST_PERCEPCION = 5         # (int) A cuántas casillas de ditancia puede un zorro detectar a un conejo. 5 casilas = 1 km
COOLDOWN_ZORRO = 1          # (int) Cuántos ticks debe esperar un zorro para poder alimentarse de nuevo
PROB_ELIM_CONEJOS = 0.02    # (float entre 0 y 1) Probabilidad de que al alimentarse, un zorro elimine a una familia de conejos
PROB_CAZAR = 0.2            # (float entre 0 y 1) Probablidad de que al estar adyacente a una familia de conejos, el zorro se alimente de ella
PROB_PELEA = 0.7            # (float entre 0 y 1) Probabilidad de que al estar adyacente a otro zorro, el zorro "actual" lo elimine

# Constantes posicionales (son todas para acceder a elementos dentro de distintas listas - son todas de tipo int)
KEY = 0             # Al obtener un animal del diccionario, indica que el primer elemento es la llave
VALUE = 1           # Al obtener un animal del diccionario, indica que el segundo elemento es el valor (la lista con sus datos)
FILA = 0            # En las llaves del diccionario y todo tipo de coordenadas, indica que el primer elemento es el número de fila
COLUMNA = 1         # En las llaves del diccionario y todo tipo de coordenadas, indica que el segundo elemento es el número de columna
TIPO_CONEJO = -1    # El tipo conejo está asociado al valor -1, este valor y el del zorro sirven para que la gráfica con plt.imshow() se vea mejor
TIPO_ZORRO = 1      # El tipo zorro está asociado al valor 1
TIPO = 0            # El primer elemento de la lista de un animal indica su tipo (si es zorro o conejo)       
TIMER = 1           # El segundo elemento de la lista de un animal indica el "timer" de reproducción (conejos) o inanición (zorros)
EDAD = 2            # El tercer elemento de la lista de un animal indica su edad (cuántos ticks ha sobrevivido)
DIR_MOV = 3         # El cuarto elemento indica su dirección de movimiento predominante

Using matplotlib backend: <object object at 0x000002394D3BAA90>


<b>Para "almacenar" a los animales:</b><br><br>
-Se utilizó un diccionario, donde las llaves son las coordenadas de los animales y el valor son los datos del mismo.<br>
-El diccionario permite verificar casillas adyacentes rápidamente (si existe la llave es que hay un animal).<br>
-También permite iterar sobre los animales existentes fácilmente, al iterar sobre las llaves.<br>

In [ ]:
dictAnimales = dict()
contadorFrames = 0
numConejos = 0
numZorros = 0
listNumConejos = []
listNumZorros = []
totalTicks = 0
cantidadConejos = []
cantidadZorros = []

def teletransportar(posicion):
    """ 
    Recibe "posicion" que sería una lista con dos coordenadas. Si las coordenadas estan fuera de la grilla, se modifican
    para que estas queden dentro, de la manera que quedarían con condiciones de borde periódicas.
    """
    posicion[0] %= GRID_SIZE
    posicion[1] %= GRID_SIZE
    return posicion

def generarMovimiento(distanciaMax):
    # Se generan dos números enteros en el intervalo [-distanciaMax,distanciaMax], que sería hacia dónde se mueve un animal
    mov = [0,0]
    mov[random.randint(0,1)] = distanciaMax
    if mov[0] == 0:
        mov[1] = random.randint(-distanciaMax,distanciaMax)
    elif mov[1] == 0:
        mov[0] = random.randint(-distanciaMax,distanciaMax)
    return mov

def moverAnimal(animal, distancia = 1, random = True):
    """
    El movimiento de los animales puede ser aleatorio (si random es True) o definido por el elemento de la lista del animal "DIR_MOV".
    El elemento "DIR_MOV" en principio sólo lo tendrían los zorros, pero si se agrega la capacidad de los conejos de escaparse,
    puede ser usado por ellos también.
    Si se define una distancia, sólo se utiliza para el caso en que el movimiento sea aleatorio. 
    """

    global dictAnimales
    if not dictAnimales.get(animal): # quizá el animal que se quiere mover ya está muerto, en cuyo caso, se retorna
        return
    posicion = [animal[0],animal[1]] #suponiendo que "animal" es una tupla que indica su posicion
    if random:
        mov = generarMovimiento(distancia)
    else:
        mov = dictAnimales[animal][DIR_MOV]
    posicion = teletransportar(list(np.add(posicion,mov)))
    if not dictAnimales.get((posicion[0],posicion[1])): # si no hay animal en esa posicion
        dictAnimales[posicion[0],posicion[1]] = dictAnimales[animal]
        del dictAnimales[animal]
    return posicion

def detectarPresa(animal):
    # Recorre iterativamente las casillas a 1 de distancia primero, luego los que están a 2, 3, ..., hasta el valor DIST_PERCEPCION
    # Termina cuando encuentra un conejo o recorre todas las casillas
    global dictAnimales
    if not dictAnimales.get(animal):
        return
    posRevisada = [-1,-1] # El valor [-1, -1] indica que no se encontraron conejos
    for k in range(1,DIST_PERCEPCION+1): # k indica a qué distancia se está revisando
        # Se parte arriba a la izquierda en cada nueva distancia [-1,-1], [-2,-2], ... (respecto a la posicion del animal)
        for i in range(-k,k+1): # i se mueve en la fila en sentido positivo (derecha), se mantiene la columna fija
            posRevisada = tuple(teletransportar([animal[0]+i,animal[1]-k]))
            if dictAnimales.get(posRevisada):
                if dictAnimales[posRevisada][TIPO] == TIPO_CONEJO:
                    return (i,-k)
        for j in range(-k+1,k+1):# j se mueve en la columna en sentido positivo (abajo), se mantiene la fila fija
            posRevisada = tuple(teletransportar([animal[0]+k,animal[1]+j]))
            if dictAnimales.get(posRevisada):
                if dictAnimales[posRevisada][TIPO] == TIPO_CONEJO:
                    return (k,j)
        for i in range(k-1,-k-1, -1): # i se mueve en la fila en sentido negativo (izquierda), se mantiene la columna fija
            posRevisada = tuple(teletransportar([animal[0]+i,animal[1]+k]))
            if dictAnimales.get(posRevisada):
                if dictAnimales[posRevisada][TIPO] == TIPO_CONEJO:
                    return (i,k)
        for j in range(k-1,-k, -1): # j se mueve en la columna en sentido negativo (arriba), se mantiene la fila fija
            posRevisada = tuple(teletransportar([animal[0]-k,animal[1]+j]))
            if dictAnimales.get(posRevisada):
                if dictAnimales[posRevisada][TIPO] == TIPO_CONEJO:
                    return (-k,j)
    return [-1,-1]


def crearAnimalesInic():
    # Inicializar el diccionario con una cantidad de conejos y zorros definida con parámetros globales
    global dictAnimales
    global numConejos
    global numZorros
    # Generar conejos
    for x in range(0,NUM_INICIAL_CONEJOS):
        pos = [random.randint(0,GRID_SIZE-1),random.randint(0,GRID_SIZE-1)]
        while dictAnimales.get((pos[0],pos[1])): #si ya existe la llave en el diccionario, busca una que no exista
            pos = [random.randint(0,GRID_SIZE-1),random.randint(0,GRID_SIZE-1)]
        dictAnimales[pos[0],pos[1]] = [TIPO_CONEJO, 0, 0]
    numConejos += NUM_INICIAL_CONEJOS
    # Generar zorros
    for x in range(0,NUM_INICIAL_ZORROS):
        pos = [random.randint(0,GRID_SIZE-1),random.randint(0,GRID_SIZE-1)]
        while dictAnimales.get((pos[0],pos[1])): #si ya existe la llave en el diccionario, busca una que no exista
            pos = [random.randint(0,GRID_SIZE-1),random.randint(0,GRID_SIZE-1)]
        dictAnimales[pos[0],pos[1]] = [TIPO_ZORRO, 0, 0, generarMovimiento(DIST_MOV_ZORRO)]
    numZorros += NUM_INICIAL_ZORROS
    listNumConejos.append(numConejos)
    listNumZorros.append(numZorros)



<b>Loop principal:</b><br><br>
-Itera la cantidad de veces que se especifique en TOTAL_TICKS.<br>
-Tiene un loop secundario que recorre a todos los animales.<br>
-En el loop secundario se verifican muertes, reproducción, alimentación y se determinan (y llevan a cabo) movimientos.<br>
-Si en algún momento se extinguen los conejos o zorros, se genera uno nuevo de estos (no hay extinción total).

In [ ]:
#print(dictAnimales)
crearAnimalesInic()

# LOOP PRINCIPAL, cada iteración es un tick
for i in range(0,TOTAL_TICKS):
    if VISUALIZAR: grilla = np.full((GRID_SIZE,GRID_SIZE),0)
    #if i % 100 == 99:
    if i % 200 == 0:
        print("# ticks: ",i," conejos: ", numConejos," zorros: ",numZorros)
    animales = list(dictAnimales.items()).copy()
    for animal in animales:
        #---Verificar que animal exista en diccionario---
        if not dictAnimales.get(animal[KEY]):
            continue
        posicion = animal[KEY]
        if VISUALIZAR: grilla[posicion[FILA]][posicion[COLUMNA]] = dictAnimales[posicion][TIPO]
        #---Paso de tiempo (aumento de edad y tiempo desde alimentacion/reproduccion)---
        dictAnimales[posicion][EDAD] += 1
        dictAnimales[posicion][TIMER] += 1
        #---Verificar tipo de animal---
        if dictAnimales[posicion][TIPO] == TIPO_CONEJO:
            #---Verificar muerte por edad---
            if dictAnimales[posicion][EDAD] > VIDA_CONEJO:
                del dictAnimales[posicion]
                numConejos -= 1
                continue
            #---Reproduccion de conejos---
            if dictAnimales[posicion][TIPO] == TIPO_CONEJO and dictAnimales[posicion][TIMER] > FREC_REP_CONEJO:
                posAux = [posicion[FILA],posicion[COLUMNA]].copy()
                posNueva = [0,0]
                if random.randint(1,100)<=PROB_REP_LEJANA*100:
                    posNueva = moverAnimal(posicion,DIST_REP_CONEJO*3,True)
                else:
                    posNueva = moverAnimal(posicion,DIST_REP_CONEJO,True)
                if not dictAnimales.get((posAux[0],posAux[1])):
                    # Si hay un animal en la posicion que resultó aleatoriamente, no se produce reproducción
                    nuevoAni = [TIPO_CONEJO,random.randint(-int(FREC_REP_CONEJO/5),int(FREC_REP_CONEJO/5)),0] #para que la reproduccion se mas heterogenea
                    dictAnimales[posAux[0],posAux[1]] = nuevoAni
                    dictAnimales[(posNueva[0],posNueva[1])][TIMER] = 0
                    numConejos += 1
            else:
            #---Mover al conejo---
                posAux = moverAnimal(posicion)
        elif dictAnimales[posicion][TIPO] == TIPO_ZORRO:
            #---Verificar muerte por edad o inanicion---
            if dictAnimales[posicion][EDAD] > VIDA_ZORRO or dictAnimales[posicion][TIMER] > FREC_ALI_ZORRO:
                del dictAnimales[posicion]
                numZorros -= 1
                continue
            #---Detectar presa, si es que zorro no está en cooldown---
            if dictAnimales[posicion][TIMER] > COOLDOWN_ZORRO:
                dirPresa = list(detectarPresa(posicion))
                if dirPresa != [-1,-1]:
                    if abs(dirPresa[0]) > DIST_MOV_ZORRO:
                        dirPresa[0] = int(dirPresa[0]/abs(dirPresa[0]))*DIST_MOV_ZORRO
                    else:
                        #la idea es que el zorro quede al lado de la presa y no en la misma casilla:
                        if dirPresa[0] < 0:
                            dirPresa[0] += 1 
                        elif dirPresa[0] > 0:
                            dirPresa[0] -= 1
                    if abs(dirPresa[1]) > DIST_MOV_ZORRO:
                        dirPresa[1] = int(dirPresa[1]/abs(dirPresa[1]))*DIST_MOV_ZORRO
                    else:
                        if dirPresa[1] < 0:
                            dirPresa[1] += 1
                        elif dirPresa[1] > 0:
                            dirPresa[1] -= 1
                    dictAnimales[posicion][DIR_MOV] = dirPresa
                if dictAnimales[posicion][DIR_MOV] == [0,0]:
                    # El zorro se queda cerca de su posición hasta que pueda alimentarse de nuevo
                    dictAnimales[posicion][DIR_MOV] = generarMovimiento(DIST_MOV_ZORRO)
            
            #---Mover al zorro---
            if random.randint(0,100) <= PROB_MOV_RAND*100:
                # Movimiento aleatorio
                posAux = moverAnimal(posicion,1,True)
            else:
                # Movimiento predominante
                posAux = moverAnimal(posicion,DIST_MOV_ZORRO,False)
        posNueva = (posAux[0],posAux[1])
        #---Zorro se alimenta---
        if dictAnimales[posNueva][TIPO] == TIPO_ZORRO and dictAnimales[posNueva][TIMER] > COOLDOWN_ZORRO:
            seAlimento = False
            # revisa casillas adyacentes, se come un sólo conejo:
            for fila in range(-1,2):
                if seAlimento: break
                for col in range(-1,2):
                    if seAlimento: break
                    # con "teletransportar" se asegura que las casillas revisadas estén dentro de la grilla:
                    posRevisada = teletransportar([posNueva[FILA] + fila, posNueva[COLUMNA] + col])
                    if dictAnimales.get((posRevisada[0],posRevisada[1])):
                        animalCerca = dictAnimales[posRevisada[0],posRevisada[1]]
                        if animalCerca[TIPO] == TIPO_CONEJO:
                            # Se hacen randoms entre 0 y 1000 para tener un control más fino usando randint (en vez de hacer 0-100):
                            if random.randint(1,1000)<=PROB_CAZAR*1000:
                                # El zorro se alimenta con cierta probabilidad
                                dictAnimales[posNueva][TIMER] = 0
                                dictAnimales[posNueva][DIR_MOV] = [0,0]
                                #---Reproduccion de zorro---
                                if random.randint(1,1000)<=PROB_REP_ZORRO*1000:
                                    # Se reproduce con cierta probabilidad
                                    nuevoAni = [TIPO_ZORRO,0,0,generarMovimiento(DIST_MOV_ZORRO)]
                                    dictAnimales[posRevisada[0],posRevisada[1]] = nuevoAni                    
                                    numZorros += 1
                                    numConejos -= 1
                                else:
                                    # Se elimina familia de conejos con cierta probabilidad
                                    if random.randint(1,1000)<=PROB_ELIM_CONEJOS*1000:
                                        del dictAnimales[posRevisada[0],posRevisada[1]]
                                        numConejos -= 1
                                seAlimento = True
                        elif animalCerca[TIPO] == TIPO_ZORRO and [fila,col]!=[0,0]:
                            #---Pelea de zorros: se eliminan zorros adyacentes con cierta probabilidad---
                            if random.randint(1,1000)<=PROB_PELEA*1000:
                                del dictAnimales[posRevisada[0],posRevisada[1]]
                                numZorros -= 1

    listNumConejos.append(numConejos)
    listNumZorros.append(numZorros)
    # En caso de extinción: generar un nuevo zorro en (0,0) o nuevo conejo en la mitad de la grilla (aprox)
    if numZorros <= 0:
        zorro = [TIPO_ZORRO,0,0,[random.randint(-DIST_MOV_ZORRO,DIST_MOV_ZORRO),random.randint(-DIST_MOV_ZORRO,DIST_MOV_ZORRO)]]
        if dictAnimales.get((int(GRID_SIZE/2),int(GRID_SIZE/2))):
            del dictAnimales[int(GRID_SIZE/2),int(GRID_SIZE/2)]
            numConejos -= 1 # a veces puede pasar que se elimine un conejo al poner un nuevo zorro
        dictAnimales[int(GRID_SIZE/2),int(GRID_SIZE/2)] = zorro
        numZorros += 1
        if VISUALIZAR: grilla[int(GRID_SIZE/2)][int(GRID_SIZE/2)] = TIPO_ZORRO 
    if numConejos <= 0: 
        numConejos=1
        conejo = [TIPO_CONEJO,0,0]
        if dictAnimales.get((0,0)):
            del dictAnimales[0,0]
            numZorros -= 1 # a veces puede pasar que se elimine un zorro al poner un nuevo conejo
        dictAnimales[0,0] = conejo
        if VISUALIZAR: grilla[0,0] = TIPO_CONEJO
    # Se muestra la grilla en pantalla si visualizar es True
    if VISUALIZAR:
        plt.clf()
        plt.imshow(grilla, cmap="bwr")
        plt.pause(TICK_RATE)  

# ticks:  0  conejos:  200  zorros:  1
# ticks:  200  conejos:  186  zorros:  4
# ticks:  400  conejos:  144  zorros:  13
# ticks:  600  conejos:  213  zorros:  16
# ticks:  800  conejos:  200  zorros:  15
# ticks:  1000  conejos:  243  zorros:  18
# ticks:  1200  conejos:  246  zorros:  17
# ticks:  1400  conejos:  265  zorros:  25
# ticks:  1600  conejos:  245  zorros:  22
# ticks:  1800  conejos:  202  zorros:  27
# ticks:  2000  conejos:  188  zorros:  14


In [ ]:
# Graficar las poblaciones de zorros y conejos:
ticks = list(range(0,TOTAL_TICKS+1))
plt.ion()
fig, ax1 = plt.subplots()
ax1.set_xlabel('Ticks') 
ax1.set_ylabel('Numero de Conejos') 
ax1.plot(ticks, listNumConejos, color = 'blue')
ax2 = ax1.twinx() 
ax2.plot(ticks, listNumZorros , color = 'orange') 
ymin, ymax = plt.ylim()
scale_factor = 3
plt.ylim(ymin * scale_factor, ymax * scale_factor)
plt.ylabel('Numero de Zorros') 
plt.show()
plt.pause(1)